In [2]:
!pip install lxml
!pip install spacy

In [3]:
from lxml import etree
import spacy
import pandas as pd

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/Commedia_Petrocchi_XML_TEI.xml'

In [6]:
parser = etree.XMLParser(recover=True)
xml_data = etree.parse(file_path, parser)

In [7]:
cantica = xml_data.xpath('//div1/@id')[0]
tot_canti = xml_data.xpath('//div1[@id="Inferno"]//div2')

In [8]:
cantiche = xml_data.xpath('//div1')  # Estrae tutte le cantiche
canti = {}

# Itera su ogni cantica
for cantica_element in cantiche:
    cantica = cantica_element.xpath('./@id')[0]  # Estrae il nome della cantica (es. "Inferno", "Purgatorio", "Paradiso")
    tot_canti = cantica_element.xpath('./div2')  # Estrae tutti i canti per la cantica corrente

    # Processa i canti e crea un dizionario con le terzine come stringhe
    for c in tot_canti:
        h_canto = c.xpath('.//head//text()')  # Estrae il titolo del canto
        terzine = []

        for lg in c.xpath('.//lg'):
            # Ogni verso viene unito in una stringa con uno spazio come separatore
            terzina = [l.text.strip() for l in lg.xpath('./l') if l.text.strip()]
            terzine.append(terzina)

        # Aggiunge il canto alla struttura canti
        canti[(cantica, h_canto[0])] = terzine

# Genera il risultato finale
data = []
for (cantica, titolo), ter in canti.items():
    i = 1
    for t in ter:
        # Controlla il numero di versi nella terzina
        if len(t) == 3:  # Se è una terzina
            range_versi = f"{i}-{i+2}"  # Costruisce il range per una terzina
            frase = " ".join(t)  # Combina i versi in una stringa unica
            data.append([cantica, titolo, range_versi, frase])
            i += 3
        else:  # Se è un singolo verso o gruppo di versi non standard
            for verso in t:
                range_versi = f"{i}"  # Singolo verso
                data.append([cantica, titolo, range_versi, verso])
                i += 1


df = pd.DataFrame(data, columns=['cantica', 'canto', 'range_versi', 'terzina'])
df['range_versi'] = "\u200B" + df['range_versi'].astype(str)
df.to_csv("terzine.csv", index=False, sep=';', encoding='utf-8-sig')
print(df)

       cantica   canto range_versi  \
0      Inferno       I        ​1-3   
1      Inferno       I        ​4-6   
2      Inferno       I        ​7-9   
3      Inferno       I      ​10-12   
4      Inferno       I      ​13-15   
...        ...     ...         ...   
4829  Paradiso  XXXIII    ​133-135   
4830  Paradiso  XXXIII    ​136-138   
4831  Paradiso  XXXIII    ​139-141   
4832  Paradiso  XXXIII    ​142-144   
4833  Paradiso  XXXIII        ​145   

                                                terzina  
0     Nel mezzo del cammin di nostra vita mi ritrova...  
1     Ahi quanto a dir qual era è cosa dura esta sel...  
2     Tant'è amara che poco è più morte; ma per trat...  
3     Io non so ben ridir com'i' v'intrai, tant'era ...  
4     Ma poi ch'i' fui al piè d'un colle giunto, là ...  
...                                                 ...  
4829  Qual è 'l geomètra che tutto s'affige per misu...  
4830  tal era io a quella vista nova: veder voleva c...  
4831  ma non eran da 